<a href="https://colab.research.google.com/github/stereifberger/logical_derivations_with_transformers/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone whole github repository with necessary files
!git clone https://github.com/stereifberger/logical_derivations_with_transformers
!cd /content/logical_derivations_with_transformers

Cloning into 'logical_derivations_with_transformers'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 61 (delta 24), reused 46 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 131.03 KiB | 16.38 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
# Import libraries
import sys
sys.path.append('logical_derivations_with_transformers')
from imports import *

In [3]:
importlib.reload(train_test)

<module 'train_test' from '/content/logical_derivations_with_transformers/train_test.py'>

In [3]:
# Set random seed for reproducibility
seed(42)
torch.manual_seed(42)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Prepare data
total_samples = 10000
#batch_size = 32
data_inputs, data_targets, max_input_len, max_target_len = utils.prepare_data(num_samples=total_samples)
t_nu = 5
vocab_size = t_nu + 11  # Number of symbols including PAD, SOS, EOS
inputs_train, inputs_test, targets_train, targets_test = train_test_split(
    data_inputs, data_targets, test_size=0.2, random_state=42
)

# Construct data loader
#train_data = list(zip(inputs_train, targets_train))
#test_data = list(zip(inputs_test, targets_test))
#train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=train_test.collate_batch)
#test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=train_test.collate_batch)

In [6]:
import string

alphabet = string.ascii_lowercase  # Get lowercase letters 'a', 'b', 'c', ...

def convert_to_letters(data):
  """Converts a list of lists of integers to letters.

  Args:
    data: A list of lists of integers.

  Returns:
    A list of lists of letters.
  """
  converted_data = []
  for sublist in data:
    converted_sublist = [alphabet[i] for i in sublist]  # Map integers to letters
    converted_data.append(converted_sublist)
  return converted_data

# Convert the data
inputs_train = convert_to_letters(inputs_train)
inputs_test = convert_to_letters(inputs_test)
targets_train = convert_to_letters(targets_train)
targets_test = convert_to_letters(targets_test)

# Print an example to verify
print(inputs_train[0])  # Print the first element of inputs_train_letters

['c', 'h', 'b', 'k', 'e', 'i', 'g', 'h', 'c', 'l', 'h', 'h', 'b', 'k', 'e', 'i', 'k', 'c', 'i', 'i', 'p', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


# Gemma

In [7]:

!pip install transformers accelerate datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset

In [9]:
# Check for avaiable GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("Using CPU")

Using CUDA


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "KingNish/Qwen2.5-0.5b-Test-ft"
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device) # Use AutoModelForCausalLM instead of AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# prompt: Load train and test data as well as the model to the gpu

import torch

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example of moving a tensor to the GPU
# Assuming 'inputs_train' is a list of lists of strings, convert to tensors first
# This is an example and may not directly apply without knowing the data format
# Replace with appropriate conversion to tensor if needed
# example_tensor = torch.tensor(inputs_train[0])
# example_tensor = example_tensor.to(device)


# Example of moving training and test data to GPU
# Assuming 'inputs_train' etc are lists of lists.  You will need to adapt this to the actual format of your data.
# Function to move data to GPU (modify as needed based on data structure)
def move_to_gpu(data):
    # Assuming data is list of strings, tokenize it
    tokenized_data = [tokenizer(item, return_tensors="pt").input_ids.to(device) for item in data]
    return tokenized_data

inputs_train_gpu = move_to_gpu(inputs_train)
inputs_test_gpu = move_to_gpu(inputs_test)
targets_train_gpu = move_to_gpu(targets_train)
targets_test_gpu = move_to_gpu(targets_test)


print("Model and data moved to:", device)


In [10]:
train_dataset = Dataset.from_dict({"input": inputs_train, "target": targets_train})
test_dataset = Dataset.from_dict({"input": inputs_test, "target": targets_test})

In [13]:
# prompt: Preprocess train_dataset and test_dataset

def preprocess_function(examples):
    inputs = ["".join(x) for x in examples["input"]]
    targets = ["".join(x) for x in examples["target"]]
    # Use the same max_length for both input and target
    max_len = max(max_input_len, max_target_len)
    model_inputs = tokenizer(inputs, padding="max_length", max_length=max_len, truncation=True).to(device)
    labels = tokenizer(targets, padding="max_length", max_length=max_len, truncation=True).to(device)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

test_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names,
)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
training_args = TrainingArguments(
       output_dir="./results",
       per_device_train_batch_size=32,
       per_device_eval_batch_size=32,
       num_train_epochs=20,
       report_to="tensorboard",
   )

In [ ]:
# prompt: Train the model using trainer and track and plot the train and testloss and train and test accuracy.

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

train_results = trainer.train()
trainer.save_model("./results")
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(eval_dataset=test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Plotting
import matplotlib.pyplot as plt

train_loss = [log["loss"] for log in trainer.state.log_history if "loss" in log]
train_accuracy = [log["eval_accuracy"] if "eval_accuracy" in log else 0 for log in trainer.state.log_history ]
epochs = range(1, len(train_loss) + 1)

eval_loss = [metrics["eval_loss"]] if "eval_loss" in metrics else [0]
eval_accuracy = [metrics["eval_accuracy"]] if "eval_accuracy" in metrics else [0]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot([len(train_loss)], eval_loss, marker='o', markersize=8, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot([len(train_accuracy)], eval_accuracy, marker='o', markersize=8, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Step,Training Loss


In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.04704412460327148, metrics={'train_runtime': 81.431, 'train_samples_per_second': 39.297, 'train_steps_per_second': 1.228, 'total_flos': 666549443788800.0, 'train_loss': 0.04704412460327148, 'epoch': 20.0})

In [37]:
from operator import index
for i in range(min(50, len(test_dataset))):  # Use min to avoid IndexError if test_dataset has fewer than 50 entries
    example = test_dataset[i]
    inputs = example['input_ids']
    labels = example['labels']  # Access labels directly

    # Assuming your model is called 'model' and is already loaded
    with torch.no_grad():
        # Reshape labels to have batch size of 1 and move to device
        outputs = model(labels.unsqueeze(0).to(device)).logits
        # Note: Providing labels as input for teacher forcing during inference

    # Process the model outputs
    predicted_ids = torch.argmax(outputs, dim=-1)
    predicted_tokens = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

    #reverse_inputs = "".join([symb_reverse[i] for i in tokenizer.decode(inputs)])
    reverse_inputs = tokenizer.decode(inputs)
    index = reverse_inputs.find("a")
    reverse_inputs = "".join([symb_reverse[i] for i in reverse_inputs[:index]])

    reverse_target = tokenizer.decode(labels)
    index = reverse_target.find("a")
    reverse_target = "".join([symb_reverse[i] for i in reverse_target[:index]])

    pred = predicted_tokens[0]
    index = pred.find("a")
    reverse_pred = "".join([symb_reverse[i] for i in pred[:index]])

    print(f"Input {i+1}: {reverse_inputs}") # Decode the input sequence
    print(f"Target {i+1}: {reverse_target}")
    print(f"Output {i+1}: {reverse_pred}") # Print the decoded prediction
    print("-" * 20)

Input 1: (((t→r)→t)∧((p∨s)→s))⊢((((t→r)→t)∧((p∨s)→s))∨((q∨p)∧q))
Target 1: (((t→r)→t)∧((p∨s)→s))((((t→r)→t)∧((p∨s)→s))∨((q∨p)∧q))
Output 1: (((→rq)tt((t)∧)∨((((t→rttp∨s((p∨s))t)∨((q∨p)→tq)tt))))))
--------------------
Input 2: (p∧p)⊢((p∧p)→p)
Target 2: (p∧p)p((p∧p)→p)
Output 2: (()()p)()→∧pp)→)p
--------------------
Input 3: (s∧q)p⊢(p→(r∨p))
Target 3: (s∧q)p(r∨p)(p→(r∨p))
Output 3: (s∧q)pq)qpt→()))p)
--------------------
Input 4: ((r∨(t∨p))∨t)⊢(((r∨(t∨p))∨t)∨(q∧s))
Target 4: ((r∨(t∨p))∨t)(((r∨(t∨p))∨t)∨(q∧s))
Output 4: (q)s∧st∧t∨(tt∨(())t∨t∧t)∧p)∧(q∧s)r∨((t)
--------------------
Input 5: ((q∨(t→r))∨(q∧s))⊢(((q∨(t→r))∨(q∧s))∧(p∨((q∨(t→r))∨(q∧s))))
Target 5: ((q∨(t→r))∨(q∧s))(p∨((q∨(t→r))∨(q∧s)))(((q∨(t→r))∨(q∧s))∧(p∨((q∨(t→r))∨(q∧s))))
Output 5: (t→(r→((((q((t→((t→(t→t((q∧s∧s(t→(((t→t(q∧s∧st)∧t→p∨t→(t→t(q(q∧s)∧(((t→t→r)r)
--------------------
Input 6: q⊢(q→(((q→s)→¬t)∨q))
Target 6: q(((q→s)→¬t)∨q)(q→(((q→s)→¬t)∨q))
Output 6: ((((q→s)→q()→(q(s)∨))))())t)∨qq)((t)∨t)∨
--------------------


In [29]:
# Reverse mapping from numerical values to symbols
symb_reverse = {"a": "",
                "b": "p",
                "c": "q",
                "d": "r",
                "e": "s",
                "f": "t",
                "g": "⊢",
                "h": "(",
                "i": ")",
                "j": "¬",
                "k": "→",
                "l": "∨",
                "m": "∧",
                "n": "⊥",
                "o": "",
                "p": "",
                }